In [46]:
import subprocess
import pandas as pd

from os import walk
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import AlignIO
from CAI import CAI

from tqdm.notebook import tqdm

In [47]:
WORK_PATH = 'aligments_and_sequenses_separated_genes/proteins'
MACSE_PATH = '/Users/pitikov_egor/Documents/dora_tree_final/macse_v2.06.jar'
PROTEIN_SET = ['atp6', 'co1', 'co2', 'co3', 'cob', 'nad1', 'nad2', 'nad3', 'nad4', 'nad4l', 'nad5', 'nad6']

In [75]:
def make_gc_table(ids, path):
    t = pd.DataFrame(ids)
    t.loc[:, 'gc'] = 5
    t.to_csv(path, sep = '\t', index = False, header = False)
    return path

def make_ali_prot(fasta_path, fasta_alias):
    d_list = []
    for rec in SeqIO.parse(f'{fasta_path}/seqs/nt/{fasta_alias}.fasta', 'fasta'):
        d_list.append(rec.description)
        
    gct = make_gc_table(d_list, f'{fasta_path}/seqs/nt/{fasta_alias}_gc.tsv')
    
    cmd = f'java -jar {MACSE_PATH} -prog alignSequences -gc_file {gct} -out_AA {fasta_path}/ali/aa/{fasta_alias}_ali_aa_codone.fasta -out_NT {fasta_path}/ali/nt/{fasta_alias}_ali_nt_codone.fasta -seq {fasta_path}/seqs/nt/{fasta_alias}.fasta'
    p = subprocess.Popen(cmd, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    out, err = p.communicate()
            
    cmd = f'muscle -in {fasta_path}/seqs/nt/{fasta_alias}.fasta -out {fasta_path}/ali/nt/{fasta_alias}_ali_nt.fasta'
    p = subprocess.Popen(cmd, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    out, err = p.communicate()

    cmd = f'muscle -in {fasta_path}/seqs/aa/{fasta_alias}.fasta -out {fasta_path}/ali/aa/{fasta_alias}_ali_aa.fasta'
    p = subprocess.Popen(cmd, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    out, err = p.communicate()
    return

In [76]:
for prot_name in tqdm(PROTEIN_SET, desc = 'Progress in proteins...'):
    make_ali_prot(WORK_PATH, prot_name)

Progress in proteins...:   0%|          | 0/12 [00:00<?, ?it/s]

In [77]:
def replase_smbs(file, smb1, smb2):
    with open(file, 'r') as read_f:
        t = read_f.read()
    with open(file, 'w') as file_write:
        file_write.write(t.replace(smb1, smb2))

In [78]:
for prot_name in tqdm(PROTEIN_SET, desc = 'Progress in proteins...'):
    replase_smbs(f'{WORK_PATH}/ali/aa/{prot_name}_ali_aa_codone.fasta', '!', '-')
    replase_smbs(f'{WORK_PATH}/ali/nt/{prot_name}_ali_nt_codone.fasta', '!', '-')

Progress in proteins...:   0%|          | 0/12 [00:00<?, ?it/s]

In [148]:
def unite_fasta(fasta_set, fasta_output):
    res = {}
    tresholds = [0]
    for filepath in fasta_set:
        orgs = {}
        for rec in SeqIO.parse(filepath, 'fasta'):
            orgname = ' '.join(rec.description.split(' ')[: -1])
            orgs[orgname] = str(rec.seq)
        for i in set(orgs.keys()).intersection(set(res.keys())):
            res[i] += orgs[i]
        for i in orgs.keys() - res.keys():
            res[i] = '-' * sum(tresholds) + orgs[i]
            print(f'NEW ORGANISM {i} ON {filepath.split("/")[-1]} file')
        for i in res.keys() - orgs.keys():
            res[i] += '-' * tresholds[-1]
            print(f'OLD ORGANISM {i} HAVE NOT BEEN MET IN {filepath.split("/")[-1]} file')
            
        tresholds.append(len(orgs[orgname]))
    return res, tresholds
            

In [149]:
for prot_name in PROTEIN_SET:
    replase_smbs(f'{WORK_PATH}/ali/aa/{prot_name}_ali_aa_codone.fasta', '!', '-')
    replase_smbs(f'{WORK_PATH}/ali/nt/{prot_name}_ali_nt_codone.fasta', '!', '-')
    
nt_set_codone = [f'{WORK_PATH}/ali/nt/{prot_name}_ali_nt_codone.fasta' for prot_name in PROTEIN_SET]
aa_set_codone = [f'{WORK_PATH}/ali/aa/{prot_name}_ali_aa_codone.fasta' for prot_name in PROTEIN_SET]

nt_set = [f'{WORK_PATH}/ali/nt/{prot_name}_ali_nt.fasta' for prot_name in PROTEIN_SET]
aa_set = [f'{WORK_PATH}/ali/aa/{prot_name}_ali_aa.fasta' for prot_name in PROTEIN_SET]

print('''
--------------------------------------
------START NT CODONE ----------------
--------------------------------------
''')
  = unite_fasta(nt_set_codone, f'{WORK_PATH}/ali/nt/total_ali_nt_codone.fasta')
print('''
--------------------------------------
------START AA CODONE ----------------
--------------------------------------
''')
aa_codone, aa_c_tr = unite_fasta(aa_set_codone, f'{WORK_PATH}/ali/aa/total_ali_aa_codone.fasta')
print('''
--------------------------------------
------START NT -----------------------
--------------------------------------
''')
nt, nt_tr = unite_fasta(nt_set, f'{WORK_PATH}/ali/nt/total_ali_nt.fasta')
print('''
--------------------------------------
------START AA -----------------------
--------------------------------------
''')
aa, aa_tr = unite_fasta(aa_set, f'{WORK_PATH}/ali/aa/total_ali_aa.fasta')


--------------------------------------
------START NT CODONE ----------------
--------------------------------------

NEW ORGANISM  MW528029.1 Boccardiella hamata ON atp6_ali_nt_codone.fasta file
NEW ORGANISM  MK120303.1 Marenzelleria neglecta ON atp6_ali_nt_codone.fasta file
NEW ORGANISM  OQ078742.1 Polydora cf. ciliata DS-2023 ON atp6_ali_nt_codone.fasta file
NEW ORGANISM  NC_028711.1 Magelona mirabilis ON atp6_ali_nt_codone.fasta file
NEW ORGANISM  NC_061377.1 Polydora hoplura ON atp6_ali_nt_codone.fasta file
NEW ORGANISM  MW316633.1 Polydora websteri ON atp6_ali_nt_codone.fasta file
NEW ORGANISM  OK032597.1 Lindaspio sp. i ZZ-2021 ON atp6_ali_nt_codone.fasta file
NEW ORGANISM  MW316635.1 Polydora brevipalpa ON atp6_ali_nt_codone.fasta file

--------------------------------------
------START AA CODONE ----------------
--------------------------------------

NEW ORGANISM  MW528029.1 Boccardiella hamata ON atp6_ali_aa_codone.fasta file
NEW ORGANISM  MK120303.1 Marenzelleria neglecta 

In [217]:
def make_fasta_and_nexus(ali_dict, 
                         tresholds, 
                         outdir, 
                         alias, 
                         feat_type = 'DNA', 
                         model = 'GTR+I+G'):
    to_file = [SeqRecord(id = '', description = i, seq = Seq(ali_dict[i])) for i in ali_dict.keys()] 
    with open(f'{outdir}/{alias}.fasta', 'w') as file:
        SeqIO.write(to_file, file, 'fasta')
    feat_num = len(tresholds)
    with open(f'{outdir}/{alias}.nex', 'w') as file:
        file.write('#nexus\n')
        file.write('begin sets;\n')
        last_en = 0
        for pos, val in enumerate(tresholds, start = 1):
            if pos<feat_num:
                st = 1 + last_en
                en = tresholds[pos] + st - 1
                file.write(f'\tcharset part{pos} = {st}-{en};\n')
                last_en = en
        file.write(f'\tcharpartition mine =')
        for i in range(1, pos-1):
            file.write(f' {model}:part{i},')
        file.write(f' {model}:part{pos-1};\n')
        file.write('end;')
    return f'{outdir}/{alias}.fasta', f'{outdir}/{alias}.nex'

def make_tree(ali_dict, 
              tresholds, 
              outdir, 
              alias, 
              feat_type = 'DNA', 
              model = 'GTR+I+G', 
              finder = 'MFP+MERGE', 
              b = None, 
              mode = 'p'):
    fasta, nexus = make_fasta_and_nexus(ali_dict, 
                         tresholds, 
                         outdir, 
                         alias, 
                         feat_type = feat_type,
                         model = model)
    if b:
        cmd = f'iqtree -s {fasta} -st {feat_type} -m {finder} -B {b} -{mode} {nexus} --mlrate'
    else:
        cmd = f'iqtree -s {fasta} -st {feat_type} -m {finder} -{mode} {nexus} --mlrate'
    p = subprocess.Popen(cmd, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    out, err = p.communicate()
    return out, err

In [221]:
models = ['HKY', 'JC', 'F81', 'K2P', 'K3P', 'K81uf', 'TNef', 'TIM', 'TIMef', 'TVM', ', TVMef', 'SYM', 'GTR'] 
models = models + [f'{i}+I+G' for i in models]
cfeats = ['TESTONLY', 'TEST', 'MF', 'MFP']
tfeats = ['LM', 'LMRY', 'LMWS', 'LMMK', 'LMSS', '']
feats = [f'{i}+{j}' for i in cfeats for j in tfeats]
modes = ['p', 'q']

In [ ]:
import itertools as it
cmd = f'mkdir -p test_trees_params'
p = subprocess.Popen(cmd, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
out, err = p.communicate()
for model_type, finder_type, mode in tqdm(it.product(models, feats, modes)):
    cmd = f'mkdir -p test_trees_params/{model_type}_{finder_type}_{mode}'
    p = subprocess.Popen(cmd, shell = True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    out, err = p.communicate()
    out, err = make_tree(nt_codone, 
              nt_c_tr, 
              f'test_trees_params/{model_type}_{finder_type}_{mode}', 
              alias = 'codone_all', 
              model = model_type, 
              finder = finder_type, 
              b = 1500)
    out = out.decode()
    err = err.decode()
    with open(f'test_trees_params/{model_type}_{finder_type}_{mode}/out_prog.txt', 'w') as out_file:
        out_file.write(out)
    with open(f'test_trees_params/{model_type}_{finder_type}_{mode}/err_prog.txt', 'w') as err_file:
        err_file.write(err)

0it [00:00, ?it/s]

In [218]:
!iqtree --help

IQ-TREE multicore version 2.2.0.3 COVID-edition for Mac OS X 64-bit built Sep  5 2022
Developed by Bui Quang Minh, James Barbetti, Nguyen Lam Tung,
Olga Chernomor, Heiko Schmidt, Dominik Schrempf, Michael Woodhams, Ly Trong Nhan.

Usage: iqtree [-s ALIGNMENT] [-p PARTITION] [-m MODEL] [-t TREE] ...

GENERAL OPTIONS:
  -h, --help           Print (more) help usages
  -s FILE[,...,FILE]   PHYLIP/FASTA/NEXUS/CLUSTAL/MSF alignment file(s)
  -s DIR               Directory of alignment files
  --seqtype STRING     BIN, DNA, AA, NT2AA, CODON, MORPH (default: auto-detect)
  -t FILE|PARS|RAND    Starting tree (default: 99 parsimony and BIONJ)
  -o TAX[,...,TAX]     Outgroup taxon (list) for writing .treefile
  --prefix STRING      Prefix for all output files (default: aln/partition)
  --seed NUM           Random seed number, normally used for debugging purpose
  --safe               Safe likelihood kernel to avoid numerical underflow
  --mem NUM[G|M|%]     Maximal RAM usage in GB | MB | %
  --ru

In [187]:
for i in zip(nt_codone.values()):
    if set(i) == set('-'): print(i)